In [ ]:
#CNN 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

# Load the dataset
data = pd.read_csv('TGT.csv')
df = data.drop('Date', axis=1)

# Assuming 'Close' is the target and all other columns are features
X = df.drop('Adj Close', axis=1).values
y = df['Adj Close'].values.reshape(-1,1)

# Scale the features and target
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Reshape input for the CNN [samples, time_steps, features]
time_steps = 1  # Example time steps
X_train_cnn = X_train.reshape((X_train.shape[0], time_steps, X_train.shape[1]))
X_test_cnn = X_test.reshape((X_test.shape[0], time_steps, X_test.shape[1]))

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(time_steps, X_train.shape[1])))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train_cnn, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Predicting the Test set results
y_pred_scaled = model.predict(X_test_cnn)

# Calculating RMSE and MAE using the scaled data
mae_scaled = mean_absolute_error(y_test, y_pred_scaled)
rmse_scaled = np.sqrt(mean_squared_error(y_test, y_pred_scaled))

print(f'Scaled MAE: {mae_scaled}')
print(f'Scaled RMSE: {rmse_scaled}')


In [ ]:
#BiLSTM 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv('TGT.csv')

# Preprocessing
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')
data.set_index('Date', inplace=True)

# Normalize the data
scaler = MinMaxScaler()
data_normalized = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)

# Define a scaler for the target column if separate scaling is needed
scaler_target = MinMaxScaler()
data_normalized['Adj Close'] = scaler_target.fit_transform(data[['Close']])

# Create the sequences
lookback = 5
X, Y = [], []
for i in range(lookback, len(data_normalized)):
    X.append(data_normalized.iloc[i-lookback:i].values)
    Y.append(data_normalized.iloc[i]['Close'])
X, Y = np.array(X), np.array(Y)

# Split data into training and test sets
train_size = int(len(X) * 0.8)
trainX, testX = X[:train_size], X[train_size:]
trainY, testY = Y[:train_size], Y[train_size:]

# Reshape the input for LSTM compatibility
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], data_normalized.shape[1]))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], data_normalized.shape[1]))

# Define the BiLSTM model
model = Sequential()

# Add a Bidirectional LSTM layer
model.add(Bidirectional(LSTM(50, return_sequences=True, activation='relu'), input_shape=(lookback, data_normalized.shape[1])))
model.add(Bidirectional(LSTM(50, activation='relu')))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(trainX, trainY, epochs=50, batch_size=32, validation_data=(testX, testY), verbose=1)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Predict on the test data
preds = model.predict(testX)

# Calculate and print MSE and MAE using the scaled predictions and actual values
mse_scaled = mean_squared_error(testY, preds)
mae_scaled = mean_absolute_error(testY, preds)
rmse_scaled = np.sqrt(mse_scaled)

print(f'Scaled MSE: {mse_scaled}, Scaled MAE: {mae_scaled}, Scaled RMSE: {rmse_scaled}')

In [ ]:
#CNN-BiLSTM
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Conv1D, MaxPooling1D, Flatten
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv('TGT.csv')

# Preprocessing
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')
data.set_index('Date', inplace=True)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = pd.DataFrame(scaler.fit_transform(data), columns=data.columns, index=data.index)

# Create the sequences
lookback = 5
X, Y = [], []
for i in range(lookback, len(data_normalized)):
    X.append(data_normalized.iloc[i-lookback:i].values)
    Y.append(data_normalized.iloc[i]['Close'])
X, Y = np.array(X), np.array(Y)

# Split data into training and test sets
train_size = int(len(X) * 0.8)
trainX, testX = X[:train_size], X[train_size:]
trainY, testY = Y[:train_size], Y[train_size:]

# Reshape the input for CNN-BiLSTM compatibility
trainX = np.reshape(trainX, (trainX.shape[0], lookback, data_normalized.shape[1]))
testX = np.reshape(testX, (testX.shape[0], lookback, data_normalized.shape[1]))

# Define the CNN-BiLSTM model
model = Sequential()

# Add a 1D Convolutional layer with 64 filters
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(lookback, data_normalized.shape[1])))
model.add(MaxPooling1D(pool_size=1))
model.add(Bidirectional(LSTM(50, return_sequences=True, activation='relu')))
model.add(Bidirectional(LSTM(50, activation='relu')))
model.add(Flatten())
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(trainX, trainY, epochs=50, batch_size=32, validation_data=(testX, testY), verbose=1)

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Predict on the test data
preds = model.predict(testX)

# Calculate and print MSE and MAE using the scaled predictions and actual values
mse_scaled = mean_squared_error(testY, preds)
mae_scaled = mean_absolute_error(testY, preds)
rmse_scaled = np.sqrt(mse_scaled)

print(f'Scaled MSE: {mse_scaled}, Scaled MAE: {mae_scaled}, Scaled RMSE: {rmse_scaled}')
